In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def nettoyer_lat_lon(df):
    if "lat" not in df.columns or "lon" not in df.columns:
        return pd.DataFrame(columns=df.columns)  # évite l'erreur si colonnes manquantes
    df = df.copy()
    # Convertit les valeurs en float si elles sont au format texte
    df["lat"] = pd.to_numeric(df["lat"], errors="coerce")
    df["lon"] = pd.to_numeric(df["lon"], errors="coerce")
    df = df.dropna(subset=["lat", "lon"])
    return df


In [3]:
df_colleges = pd.read_csv("../csv_clean/colleges.csv")
df_lycees = pd.read_csv("../csv_clean/lycees.csv")
df_transport = pd.read_csv("../csv_clean/transports.csv")
df_logement = pd.read_csv("../csv_clean/valeurs_foncieres_2024.csv")
df_jardins = pd.read_csv("../csv_clean/parcs_jardins_communes_clean.csv")
pd.set_option('display.precision', 15)


In [4]:
# ajout temporaire pour restreindre le dataset logement
# nombre_pieces_principales >= 4
df_logement = df_logement[df_logement["nombre_pieces_principales"] >= 4]
# df_logement = df_logement[df_logement["valeur_fonciere"] > 0 and df_logement["valeur_fonciere"] < 300000]
# logement dont la valeur fonciere est inferieure a 300000
df_logement = df_logement[df_logement["valeur_fonciere"] < 300000]
# rename code_communne to insee
df_logement.rename(columns={'code_commune': 'insee', 'latitude': 'lat', 'longitude': 'lon'}, inplace=True)
df_logement.head(2)

,valeur_fonciere,adresse_numero,adresse_nom_voie,code_postal,insee,nom_commune,type_local,surface_reelle_bati,nombre_pieces_principales,surface_terrain,lon,lat
27,165000.0,431.0,bd roger salengro,69400.0,69264,villefranche-sur-saône,appartement,78.0,4.0,NaN,4.715977,46.000086000000003
45,211100.0,10.0,rue louis thevenet,69004.0,69384,lyon 4e arrondissement,appartement,62.0,4.0,NaN,4.838431,45.778483000000001


In [5]:
df_jardins.rename(columns={'code_insee': 'insee'}, inplace=True)
df_jardins.rename(columns={'latitude': 'lat', 'longitude': 'lon'}, inplace=True)

df_jardins

,adresse,commune,surface_m2,aire_jeu,eau,toilettes,insee,lon,lat
0,1 Rue Edouard Millaud,Saint-Genis-Laval,36358.949999999997090,False,True,True,69204.0,4.786552000000000,45.692937040903672
1,2 130 Avenue Georges Clemenceau,Saint-Genis-Laval,4506.039999999999964,False,False,False,69204.0,4.792575000000000,45.696546040903307
2,5110 Place des Minimes,Lyon 5e Arrondissement,3334.820000000000164,False,True,False,69385.0,4.821792696554102,45.758162063601254
3,6073 Rue Tête d'Or,Lyon 6e Arrondissement,2094.409999999999854,True,True,False,69386.0,4.852025392874896,45.766370671287561
4,8138 7 Rue du Presbytere,Lyon 8e Arrondissement,1330.869999999999891,True,False,False,69388.0,4.851963913406681,45.735282185469629
...,...,...,...,...,...,...,...,...,...
622,57232 Berges Bertha Von Suttner,Lyon 7e Arrondissement,NaN,False,False,False,69387.0,4.833131030360194,45.747998107652499
623,54003 169 Bd de la Croix Rousse,Lyon 4e Arrondissement,NaN,False,False,False,69384.0,4.833504770725742,45.774674305444400
624,52194 Place Renée Dufourt,Lyon 2e Arrondissement,NaN,False,False,False,69382.0,4.820192537913503,45.745756179340361
625,6190 Cours Lafayette,Lyon 6e Arrondissement,NaN,False,False,False,69386.0,4.865589079193895,45.763870713733439


In [6]:
import pandas as pd
import folium
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np

# --- Fonction utilitaire pour nettoyer les lat/lon ---
def nettoyer_lat_lon(df):
    if "lat" not in df.columns or "lon" not in df.columns:
        return pd.DataFrame(columns=df.columns)
    df = df.copy()
    df["lat"] = pd.to_numeric(df["lat"], errors="coerce")
    df["lon"] = pd.to_numeric(df["lon"], errors="coerce")
    df = df.dropna(subset=["lat", "lon"])
    return df

# --- Fonction de distance Haversine (en mètres) ---
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # rayon de la Terre en mètres
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi/2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(dlambda/2)**2
    return 2 * R * np.arcsin(np.sqrt(a))


# --- Liste des communes ---
communes_insee = pd.concat([
    df_colleges[['commune', 'insee']],
    df_lycees[['commune', 'insee']],
    df_transport[['commune', 'insee']] if 'commune' in df_transport.columns and 'insee' in df_transport.columns else pd.DataFrame(columns=['commune', 'insee']),
    df_logement[['commune', 'insee']] if 'commune' in df_logement.columns and 'insee' in df_logement.columns else pd.DataFrame(columns=['commune', 'insee']),
    df_jardins[['commune', 'insee']] if 'commune' in df_jardins.columns and 'insee' in df_jardins.columns else pd.DataFrame(columns=['commune', 'insee'])
]).drop_duplicates().sort_values('commune')

options_communes = ["Toutes"] + list(communes_insee['commune'])

# --- Widgets ---
dropdown_commune = widgets.Dropdown(options=options_communes, description="Commune :", value="Toutes")
checkbox_colleges = widgets.Checkbox(value=True, description="Collèges (rouge)")
checkbox_lycees = widgets.Checkbox(value=True, description="Lycées (bleu)")
checkbox_transport = widgets.Checkbox(value=False, description="Transports (orange)")
checkbox_jardins = widgets.Checkbox(value=True, description="Jardins (vert)")

# Sliders individuels pour chaque type
slider_college = widgets.IntSlider(value=1000, min=10, max=5000, step=10, description="Collèges (m)", continuous_update=False)
slider_lycee = widgets.IntSlider(value=5000, min=10, max=10000, step=10, description="Lycées (m)", continuous_update=False)
slider_transport = widgets.IntSlider(value=100, min=10, max=500, step=10, description="Transports (m)", continuous_update=False)
slider_jardin = widgets.IntSlider(value=100, min=10, max=500, step=10, description="Jardins (m)", continuous_update=False)

def afficher_carte(
    commune,
    afficher_colleges,
    afficher_lycees,
    afficher_transports,
    afficher_jardins,
    rayon_college,
    rayon_lycee,
    rayon_transport,
    rayon_jardin
):
    # --- Filtrage par commune ---
    if commune == "Toutes":
        cols = df_colleges
        lycees = df_lycees
        transports = df_transport
        logements = df_logement
        jardins = df_jardins
    else:
        insee_code = communes_insee[communes_insee['commune'] == commune]['insee'].values[0]
        cols = df_colleges[df_colleges["insee"] == insee_code]
        lycees = df_lycees[df_lycees["insee"] == insee_code]
        transports = df_transport[df_transport["insee"] == insee_code] if "insee" in df_transport.columns else pd.DataFrame()
        logements = df_logement[df_logement["insee"] == insee_code] if "insee" in df_logement.columns else pd.DataFrame()
        jardins = df_jardins[df_jardins["insee"] == insee_code] if "insee" in df_jardins.columns else pd.DataFrame()

    # Nettoyage
    cols = nettoyer_lat_lon(cols)
    lycees = nettoyer_lat_lon(lycees)
    transports = nettoyer_lat_lon(transports)
    logements = nettoyer_lat_lon(logements)
    jardins = nettoyer_lat_lon(jardins)

    # --- Filtrage des logements selon les cases cochées (logique AND stricte) ---
    logements_proches = []
    if not logements.empty:
        for _, loge in logements.iterrows():
            proche = True

            # ✅ Appliquer la contrainte SEULEMENT si la case est cochée
            if afficher_colleges:
                if cols.empty or not np.any(haversine(loge["lat"], loge["lon"], cols["lat"], cols["lon"]) <= rayon_college):
                    proche = False

            if afficher_lycees:
                if lycees.empty or not np.any(haversine(loge["lat"], loge["lon"], lycees["lat"], lycees["lon"]) <= rayon_lycee):
                    proche = False

            if afficher_transports:
                if transports.empty or not np.any(haversine(loge["lat"], loge["lon"], transports["lat"], transports["lon"]) <= rayon_transport):
                    proche = False

            if afficher_jardins:
                if jardins.empty or not np.any(haversine(loge["lat"], loge["lon"], jardins["lat"], jardins["lon"]) <= rayon_jardin):
                    proche = False

            if proche:
                logements_proches.append(loge)

    logements_proches = pd.DataFrame(logements_proches)

    # --- Centre de la carte ---
    all_points = pd.concat(
        [df for df in [cols, lycees, logements_proches, transports, jardins] if not df.empty],
        ignore_index=True
    ) if any([not df.empty for df in [cols, lycees, logements_proches, transports, jardins]]) else pd.DataFrame()

    center_lat, center_lon = (
        (all_points["lat"].mean(), all_points["lon"].mean())
        if not all_points.empty
        else (45.75, 4.85)
    )

    # --- Création de la carte ---
    m = folium.Map(location=[center_lat, center_lon], zoom_start=12, tiles="OpenStreetMap")

    # --- Collèges ---
    if afficher_colleges and not cols.empty:
        for _, row in cols.iterrows():
            folium.CircleMarker(
                location=[row["lat"], row["lon"]],
                radius=6,
                color="red",
                fill=True,
                fill_color="red",
                fill_opacity=0.8,
                popup=f"{row['nom']} (Collège)"
            ).add_to(m)

    # --- Lycées ---
    if afficher_lycees and not lycees.empty:
        for _, row in lycees.iterrows():
            folium.CircleMarker(
                location=[row["lat"], row["lon"]],
                radius=6,
                color="blue",
                fill=True,
                fill_color="blue",
                fill_opacity=0.8,
                popup=f"{row['nom']} (Lycée)"
            ).add_to(m)

    # --- Transports ---
    if afficher_transports and not transports.empty:
        for _, row in transports.iterrows():
            folium.CircleMarker(
                location=[row["lat"], row["lon"]],
                radius=6,
                color="orange",
                fill=True,
                fill_color="orange",
                fill_opacity=0.8,
                popup=row.get("nom", "Transport")
            ).add_to(m)

    # --- Jardins ---
    if afficher_jardins and not jardins.empty:
        for _, row in jardins.iterrows():
            folium.CircleMarker(
                location=[row["lat"], row["lon"]],
                radius=6,
                color="green",
                fill=True,
                fill_color="green",
                fill_opacity=0.8,
                popup=row.get("nom", "Jardin")
            ).add_to(m)

    # --- Logements proches ---
    if not logements_proches.empty:
        for _, row in logements_proches.iterrows():
            valeur = row.get("valeur_fonciere", "N/A")
            adresse_numero = row.get("adresse_numero", "")
            adresse_voie = row.get("adresse_nom_voie", "")
            popup_html = f"""
            <b>Logement</b><br>
            <b>Valeur :</b> {valeur} €<br>
            <b>Adresse :</b> {adresse_numero} {adresse_voie}
            """
            folium.CircleMarker(
                location=[row["lat"], row["lon"]],
                radius=6,
                color="purple",
                fill=True,
                fill_color="purple",
                fill_opacity=0.8,
                popup=popup_html
            ).add_to(m)
    else:
        display(widgets.HTML("<b style='color:red;'>Aucun logement ne respecte toutes les conditions.</b>"))
        
    clear_output(wait=True)
    display(m)

# --- Liaison interactive ---
out = widgets.interactive_output(
    afficher_carte,
    {
        "commune": dropdown_commune,
        "afficher_colleges": checkbox_colleges,
        "afficher_lycees": checkbox_lycees,
        "afficher_transports": checkbox_transport,
        "afficher_jardins": checkbox_jardins,
        "rayon_college": slider_college,
        "rayon_lycee": slider_lycee,
        "rayon_transport": slider_transport,
        "rayon_jardin": slider_jardin
    }
)

# --- Interface utilisateur ---
ui = widgets.VBox([
    dropdown_commune,
    widgets.HBox([checkbox_colleges, checkbox_lycees, checkbox_transport, checkbox_jardins]),
    widgets.HTML("<b>Rayons de proximité :</b>"),
    widgets.HBox([slider_college, slider_lycee, slider_transport, slider_jardin])
])
clear_output(wait=True)
display(ui, out)


Output()